---
title: Formulación general de modelos en Pyomo
subtitle: Sets, parámetros y variables indexadas
---

[![Launch on Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/boiro9/jupyterbook2_example/HEAD?labpath=022_Pyomo_ejemplo_completo.ipynb)[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/boiro9/jupyterbook2_example/blob/HEAD/022_Pyomo_ejemplo_completo.ipynb)

En este cuaderno revisaremos el ejemplo de **planificación de producción** que desarrollamos anteriormente.  
Sin embargo, esta vez mostraremos cómo las **estructuras de datos de Python**, combinadas con las **capacidades de Pyomo**, permiten construir un modelo de optimización que **escala con los datos del problema**.

Esto significa que el modelo puede adaptarse fácilmente a:
- nuevos productos,  
- precios variables, o  
- demandas cambiantes.  

A este enfoque lo denominamos **modelado basado en datos** (*data-driven modeling*).

---

## 🧩 Nuevos componentes de Pyomo

En este cuaderno introduciremos dos componentes adicionales de Pyomo:

- [`Sets`](https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html)
- [`Parameters`](https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Parameters.html)

Estos elementos permiten definir **variables y restricciones indexadas**, es decir, asociadas a conjuntos y parámetros que representan datos del problema.  

La **combinación de conjuntos e índices** resulta esencial para construir modelos **escalables, flexibles y mantenibles**, especialmente en aplicaciones de mayor tamaño o complejidad.

---


# Formulación del Problema de planificación de manera más general

Podemos identificar un **conjunto de productos** $\mathcal{P}$ y un **conjunto de recursos** $\mathcal{R}$, junto con **parámetros** que describen cómo los recursos se transforman en productos. Las variables del problema son:
- $x_r$: la cantidad utilizada del recurso $r \in \mathcal{R}$.
- $y_p$: la cantidad producida del producto $p \in \mathcal{P}$.  
Además, ambas variables están acotadas por los valores máximos disponibles o posibles:
$$
\begin{aligned}
0 \leq x_r \leq b_r & \quad \forall r \in \mathcal{R} \\
0 \leq y_p \leq b_p & \quad \forall p \in \mathcal{P}
\end{aligned}
$$

El objetivo es **maximizar el beneficio**, definido como ingresos menos costes:
$$
\max \; \text{beneficio} = 
\sum_{p\in\mathcal{P}} c_p\, y_p - \sum_{r\in\mathcal{R}} c_r\, x_r
$$

donde:
- $c_p$ es el ingreso unitario del producto $p$,  
- $c_r$ es el coste unitario del recurso $r$.  

Las **restricciones de disponibilidad de recursos** se expresan como:

$$
\sum_{p\in\mathcal{P}} a_{r,p}\, y_p \leq x_r \quad \forall r \in \mathcal{R}
$$

donde $a_{r,p}$ indica la cantidad del recurso $r$ necesaria para producir una unidad del producto $p$.

En conjunto, el modelo general queda formulado como:

$$
\begin{align}
\max \quad & \sum_{p\in\mathcal{P}} c_p\, y_p - \sum_{r\in\mathcal{R}} c_r\, x_r \\
\text{s.a.} \quad 
& \sum_{p\in\mathcal{P}} a_{r,p}\, y_p \leq x_r && \forall r \in \mathcal{R} \\
& 0 \leq x_r \leq b_r && \forall r \in \mathcal{R} \\
& 0 \leq y_p \leq b_p && \forall p \in \mathcal{P}
\end{align}
$$

Esta formulación, basada en **conjuntos** e **índices**, permite aplicar el mismo modelo a cualquier instancia del problema (independientemente del número de productos o recursos) simplemente actualizando los datos.  
Así, el modelo se convierte en una representación **escalable y orientada a los datos**, lo que constituye la base del modelado general en Pyomo.


## 1️⃣ Datos del problema

El primer paso importante a la hora de implementar el problema es tener claro los datos del problema y el formato en el que los vamos a manejar. Recordemos que las tablas de datos que teníamos del problema eran las siguientes:

| Versión | Materia prima <br> requerida | Mano de obra A <br> requerida | Mano de obra B <br> requerida | Demanda <br> de mercado | Precio |
| :-: | :-: | :-: | :-: | :-: | :-: | 
| U | 10 g | 1 h | 2 h | $\leq$ 40 unidades | 270 |
| V |  9 g | 1 h | 1 h | ilimitada | 210 |

| Recurso | Cantidad <br> disponible | Coste |
| :-: | :-: | :-: |
| Materia prima | ilimitada | 10 / g |
| Mano de obra A | 80 horas/semana | 50 / h |
| Mano de obra B | 100 horas/semana | 40 / h |

Existen distintos tipos de estructura en Python que permiten manejar este tipo de información. En este ejemplo hemos optado por usar diccionarios anidados, ya que es una de las estructuras más sencillas y fáciles de entender. En concreto definiremos:
 - Un diccionario `productos`, donde para cada versión del producto guardaremos su precio y demanda de mercado.
 - Un diccionario `recursos`, donde para cada tipo de recurso guardaremos su precio y su cantidad máxima disponible.
 - Un diccionario `procesos`, donde para cada versión de producto especcificaremos la cantidad de recurso de cada tipo que sería necesario para su producción.

```{attention} Importante
Para una mayor cantidad de datos sería aconsejable recurrir a otro tipo de objeto, como por ejemplo `dataframes` de pandas o crear una clase específica para almacenar de la forma que más nos convenga los datos, lo cual requeriría un mayor conocimiento de Python.
```


In [24]:
# Datos
productos = {
    "U": {"precio": 270, "demanda": 40},
    "V": {"precio": 210, "demanda": 1000},
}


recursos = {
    "MP": {"precio": 10, "disponible": 1000},
    "MH A": {"precio": 50, "disponible": 80},
    "MH B": {"precio": 40, "disponible": 100},
}

procesos = {
    "U": {"MP": 10, "MH A": 1, "MH B": 2},
    "V": {"MP": 9, "MH A": 1, "MH B": 1},
}


## 2️⃣ Definición de conjuntos y parámetros

A continuación, después de crear el modelo con un `ConcreteModel()`, crearemos los conjuntos necesario para definir el problema. Para ellos usaremos el componente `Set()` de Pyomo, que permite definir un conjunto de índices sobre el que se definirán las variables, parámetros, restricciones o función objetivo del problema.

In [25]:
import pyomo.environ as pyo

# Creo el modelo
model = pyo.ConcreteModel()

# Definimos los conjuntos
# $\mathcal{P}$
model.PRODUCTOS = pyo.Set(initialize=productos.keys())
# $\mathcal{R}$
model.RECURSOS = pyo.Set(initialize=recursos.keys())

:::{tip} Tip
Los conjuntos pueden definirse directamente empleando cualquier objeto iterable de Python.  
Emplear el `Set` de Pyomo permite tener más control sobre los mismos (se pueden especificar reglas de validación) y ofrece la posibilidad de realizar una serie de [operaciones](https://pyomo.readthedocs.io/en/stable/explanation/modeling/math_programming/sets.html) sobre los conjuntos (e.j., unión o intersección de conjuntos).

```python
model.PRODUCTOS = productos.keys()
model.RECURSOS = recursos.keys()
```

Para definir como conjunto una secuencia de números, Pyomo ofrece el componente `RangeSet`, al cual se le puede especificar el primer y el último elemento de la secuencia, y el tamaño del paso (por defecto es 1).  
Por ejemplo, el siguiente código crea el conjunto de números pares entre 0 y 10:

```python
model.E = pyo.RangeSet(0, 10, 2)
```
    
:::

Ahora definimos todos los **parámetros** del modelo, para lo cual usaremos el componente `Param()` de Pyomo. Los parámetros en Python se refieren a datos de entrada (constantes numéricas) que son necesarios para formular el problema.

```{tip} 
A la hora de definir los parámetros estos se pueden definir como **mutables**, lo que significa que una vez resuelto el modelo es posible cambiar sus valores y resolverlo de nuevo sin necesidad de tener que volver a crearlo de 0. Para ello, cuando se definir, basta añadir como argumento `mutable=True`.  
En nuestro problema pondremos como mutables los parámetros $b_p$ y $b_r$ referentes a la demanda de los productos y la disponibilidad de los recursos, respectivamente
```


In [26]:
#####
# Definimos los parámetros:
####
# $b_{p}$
@model.Param(model.PRODUCTOS, domain=pyo.Any, mutable=True)
def demanda(model, producto):
    return productos[producto]["demanda"]

# $b_{r}$
@model.Param(model.RECURSOS, domain=pyo.Any, mutable=True)
def disponible(model, recurso):
    return recursos[recurso]["disponible"]

# $c_{p}$
@model.Param(model.PRODUCTOS)
def cp(model, producto):
    return productos[producto]["precio"]

# $c_{r}$
@model.Param(model.RECURSOS)
def cr(model, recurso):
    return recursos[recurso]["precio"]

# $a_{r,p}$
@model.Param(model.RECURSOS, model.PRODUCTOS)
def a(model, recurso, producto):
    return procesos[producto][recurso]

## 3️⃣ Variables

Ahora creamos las variables del problema, las cuales estarán indexadas sobre los conjuntos que definimos previamente. Además, es necesario especificar las cotas asociadas a cada una de las variables, para lo cual se define una función auxiliar que para cada índice de la variable devueve una tupla con la cota inferior y la cota superior.

In [27]:
def cotas_x(model, recurso):
    return (0, model.disponible[recurso])
model.x = pyo.Var(
    model.RECURSOS, domain=pyo.NonNegativeReals, bounds=cotas_x
)

def cotas_y(model, producto):
    return (0, model.demanda[producto])
model.y = pyo.Var(
    model.PRODUCTOS, domain=pyo.NonNegativeReals, bounds=cotas_y 
)

## 4️⃣Función objetivo

In [28]:
model.ingresos = pyo.quicksum(
    model.cp[producto] * model.y[producto] for producto in model.PRODUCTOS
)
model.costes = pyo.quicksum(
    model.cr[recurso] * model.x[recurso] for recurso in model.RECURSOS
)

# Creamos la función objetivo
@model.Objective(sense=pyo.maximize)
def beneficio(model):
    return model.ingresos - model.costes

:::{tip}
Para definir la función objetivo se ha utilizado la funciónn `quicksum` de Pyomo que acepta cualquier objeto tipo generator de Python. Podría haberse empleado también la función básica `sum` de Python, aunque la anterior es más eficiente.

## 5️⃣ Restricciones

In [29]:
# Definimos las restricciones indexadas:
@model.Constraint(model.RECURSOS)
def materiales_usados(model, recurso):
    return (
        pyo.quicksum(model.a[recurso, producto] * model.y[producto] for producto in model.PRODUCTOS) <= model.x[recurso]
    )

In [30]:
model.pprint()

2 Set Declarations
    PRODUCTOS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'U', 'V'}
    RECURSOS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'MP', 'MH A', 'MH B'}

5 Param Declarations
    a : Size=6, Index=RECURSOS*PRODUCTOS, Domain=Any, Default=None, Mutable=False
        Key           : Value
        ('MH A', 'U') :     1
        ('MH A', 'V') :     1
        ('MH B', 'U') :     2
        ('MH B', 'V') :     1
          ('MP', 'U') :    10
          ('MP', 'V') :     9
    cp : Size=2, Index=PRODUCTOS, Domain=Any, Default=None, Mutable=False
        Key : Value
          U :   270
          V :   210
    cr : Size=3, Index=RECURSOS, Domain=Any, Default=None, Mutable=False
        Key  : Value
        MH A :    50
        MH B :    40
          MP :    10
    demanda : Size=2, Index=PRODUCTOS, Domain=Any, Default=None

## 6️⃣ Resolvemos y reportamos la solución

A continuación empleamos el solver `HIGHS` para resolver el problema y mostramos la solución obtenida.

In [31]:
# Resolvemos el modelo:
solver = 'appsi_highs'
SOLVER = pyo.SolverFactory(solver)
results = SOLVER.solve(model, tee=True)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 3 rows; 5 cols; 9 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+01]
  Cost   [1e+01, 3e+02]
  Bound  [4e+01, 1e+03]
  RHS    [0e+00, 0e+00]
Presolving model
3 rows, 5 cols, 9 nonzeros  0s
3 rows, 5 cols, 9 nonzeros  0s
Presolve : Reductions: rows 3(-0); columns 5(-0); elements 9(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          3     2.6000000000e+03 Pr: 0(0) 0s
Model status        : Optimal
Simplex   iterations: 3
Objective value     :  2.6000000000e+03
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00


In [32]:
# Imprimimos la solución:
print(f"Profit = {pyo.value(model.beneficio)}")

print("\nProducción:")
for producto in model.PRODUCTOS:
    print(f" {producto}  producido =  {pyo.value(model.y[producto])}")

print("\nRecursos:")
for recurso in model.RECURSOS:
    print(f" {recurso} consumido = {pyo.value(model.x[recurso])}")

Profit = 2600.0

Producción:
 U  producido =  20.0
 V  producido =  60.0

Recursos:
 MP consumido = 740.0
 MH A consumido = 80.0
 MH B consumido = 100.0


## Código completo

:::{dropdown} Código completo

```{code-block} python
# Datos
productos = {
    "U": {"precio": 270, "demanda": 40},
    "V": {"precio": 210, "demanda": None},
}


recursos = {
    "MP": {"precio": 10, "disponible": None},
    "MH A": {"precio": 50, "disponible": 80},
    "MH B": {"precio": 40, "disponible": 100},
}

procesos = {
    "U": {"MP": 10, "MH A": 1, "MH B": 2},
    "V": {"MP": 9, "MH A": 1, "MH B": 1},
}

# Creo el modelo
import pyomo.environ as pyo

model = pyo.ConcreteModel()


# Conjuntos
model.PRODUCTOS = pyo.Set(initialize=productos.keys())
model.RECURSOS = pyo.Set(initialize=recursos.keys())

# Parámetros
@model.Param(model.PRODUCTOS, domain=pyo.Any)
def demanda(model, producto):
    return productos[producto]["demanda"]

@model.Param(model.RECURSOS, domain=pyo.Any)
def disponible(model, recurso):
    return recursos[recurso]["disponible"]

@model.Param(model.PRODUCTOS)
def cp(model, producto):
    return productos[producto]["precio"]

@model.Param(model.RECURSOS)
def cr(model, recurso):
    return recursos[recurso]["precio"]

@model.Param(model.RECURSOS, model.PRODUCTOS)
def a(model, recurso, producto):
    return procesos[producto][recurso]
	
# Variables
def cotas_x(model, recurso):
    return (0, model.disponible[recurso])
model.x = pyo.Var(
    model.RECURSOS, domain=pyo.NonNegativeReals, bounds=cotas_x
)

def cotas_y(model, producto):
    return (0, model.demanda[producto])
model.y = pyo.Var(
    model.PRODUCTOS, domain=pyo.NonNegativeReals, bounds=cotas_y 
)

# Función Objetivo
model.ingresos = pyo.quicksum(
    model.cp[producto] * model.y[producto] for producto in model.PRODUCTOS
)
model.costes = pyo.quicksum(
    model.cr[recurso] * model.x[recurso] for recurso in model.RECURSOS
)

@model.Objective(sense=pyo.maximize)
def beneficio(model):
    return model.ingresos - model.costes
	
# Restricciones
@model.Constraint(model.RECURSOS)
def materiales_usados(model, recurso):
    return (
        pyo.quicksum(model.a[recurso, producto] * model.y[producto] for producto in model.PRODUCTOS) <= model.x[recurso]
    )
	
# Resolvemos el problema
solver = 'appsi_highs'
SOLVER = pyo.SolverFactory(solver)
results = SOLVER.solve(model, tee=True)

# Imprimir solución
print(f"Profit = {pyo.value(model.beneficio)}")

print("\nProducción:")
for producto in model.PRODUCTOS:
    print(f" {producto}  producido =  {pyo.value(model.y[producto])}")

print("\nRecursos:")
for recurso in model.RECURSOS:
    print(f" {recurso} consumido = {pyo.value(model.x[recurso])}")
```

:::

## 🧠 Ejercicios propuestos

:::{exercise} Función para marcar separación datos-modelo
:label: ex:pyomo2_función

Una buena práctica en la implementación de modelos de optimización es **separar los datos de entrada** de la **formulación matemática** del modelo. Esto permite resolver el mismo modelo con distintos conjuntos de datos sin modificar su estructura.

Crea una función `formulo_modelo(productos, recursos, procesos)` que reciba como argumento los 3 diccionario con los datos necesarios para definir el modelo, y que devuelva un modelo de Pyomo completamente formulado (pero sin resolver).

```python
def formulo_modelo(productos, recursos, procesos):
     model = pyo.ConcreteModel()
     # ... definición de sets, vars, objective, constraints ...
    return modelo
```
:::

In [33]:
# Tu código aquí

:::{exercise} Resolviendo con otro solver
:label: ex:pyomo2_solver

Resuelve de nuevo el problema empleando como solver **Gurobi**.

````{solution} ex:pyomo2_solver
:label: sol:pyomo2_solver
:class: dropdown

Para resolver el problema con otro solver, basta con crear un nuevo `SolverFactory` relativo al solver que nos interese.

```{code-block} python
SOLVER_GUROBI = pyo.SolverFactory('gurobi')
results = SOLVER_GUROBI.solve(model, tee=True)

# Imprimir solución
print(f"Profit = {pyo.value(model.beneficio)}")

print("\nProducción:")
for producto in model.PRODUCTOS:
    print(f" {producto}  producido =  {pyo.value(model.y[producto])}")

print("\nRecursos:")
for recurso in model.RECURSOS:
    print(f" {recurso} consumido = {pyo.value(model.x[recurso])}")
    
```

````

:::

In [34]:
# Tu código aquí

:::{exercise} Cambiando datos
:label: ex:pyomo2_datos

Imaginemos que por causas de producción de la empresa, la cantidad disponible de mano de obra B ha bajado a 80 horas/semana. ¿Cuál sería la nueva solución óptima del problema?

````{solution} ex:pyomo2_datos
:label: sol:pyomo2_datos
:class: dropdown

Teniendo en cuenta que el parámetro asociado a la cantidad disponible de los recurso se ha definido como mutable, basta con cambiar su valor y volver a resolver.

```{code-block} python
model.disponible['MH B']=80
results = SOLVER.solve(model, tee=True)

# Imprimir solución
print(f"Profit = {pyo.value(model.beneficio)}")

print("\nProducción:")
for producto in model.PRODUCTOS:
    print(f" {producto}  producido =  {pyo.value(model.y[producto])}")

print("\nRecursos:")
for recurso in model.RECURSOS:
    print(f" {recurso} consumido = {pyo.value(model.x[recurso])}")
    
```

````

:::



In [35]:
# Tu código aquí

# 🎒 El problema de la mochila: Classic Knapsack Problem
- Dado un conjunto de objetos A, con cierto peso y beneficio
- Objetivo: seleccionar un subconjunto de esos objetos
    - Maximizando el beneficio
    - Respectando el peso máximo ($W_{max}$)

Este problema puede formularse matemáticamente de la siguiente manera:
\begin{align*}
\textrm{Maximizar}\ \ \     & \sum_{i\in A}b_{i}x_{i}\\
\textrm{sujeto a}\ \ \      & \sum_{i\in A}w_{i}x_{i}\leq W_{max}\\
                 \ \ \      & x_{i}\in \{0,1\} \, \, \forall i \in A\\                     
\end{align*}

:::{exercise} Resolviendo el problema de la mochila
:label: ex:pyomo2_mochila

Formula en Pyomo el problema de la mochila y resuévelo considerando los siguientes datos:

|Objetos(A)      | Peso(w)      | Beneficio(b) |
| :------------  | :----------: | -----------: |
| Tornillo       | 5            | 8            |
| Candado        | 7            | 3            |
| Destornillado  | 4            | 6            |
| Toalla         | 3            | 11           |

Puedes considerar como estructura de los datos la siguiente:

```{code-block} python
A = ['tornillo','candado','destornillador','toalla']
b = {'tornillo': 8,
     'candado':3,
     'destornillador':6,
     'toalla':11
    }
w = {'tornillo': 5,
     'candado':7,
     'destornillador':4,
     'toalla':3
    }
W_max = 14
```

````{solution} ex:pyomo2_mochila
:label: sol:pyomo2_mochila
:class: dropdown

```{code-block} python
import pyomo.environ as pe

A = ['tornillo','candado','destornillador','toalla']
b = {'tornillo': 8,
     'candado':3,
     'destornillador':6,
     'toalla':11
    }
w = {'tornillo': 5,
     'candado':7,
     'destornillador':4,
     'toalla':3
    }
W_max = 14

# Creamos el modelo:
model = pe.ConcreteModel()

# Variables:
model.x = pe.Var(A,within=pe.Binary)

# Función objetivo:
model.benefit = pe.Objective(
    expr=sum(b[i]*model.x[i] for i in A),
    sense= pe.maximize)

# Restricción:
model.weight = pe.Constraint(
    expr=sum(w[i]*model.x[i] for i in A)<=W_max )

# Resolvemos con HIGHS:
solver = 'appsi_highs'
SOLVER = pyo.SolverFactory(solver)
result_obj = SOLVER.solve(model, tee=True)

# Imprimimos el modelo:
#model.pprint()
#model.display()

# Imprimos valores de las variables:
print("#############################################")
print("Solución de la variable x:")
for i in model.x:
  print(str(model.x[i]), model.x[i].value)
print("")
```

````

:::

In [36]:
# Tu código aquí

# 🏭 Problema de localización de almacenes
* Determinar el conjunto de $P$ almacenes de entre el conjunto total $W$ de posibles almacenes minimizando los costes de servicio de todos los clientes $C$.
* $d_{w,c}$ representa el coste de servir al cliente $c$ desde el almacén ubicado en $w$
* Variables del problema:
    * $y_{w}\in \{0,1\}$: determina si el almacén ubicado en $w$ es seleccionado o no.
    * $x_{w,c}$: porcentaje de demanda del cliente $c$ satisfecha por el almacén $w$.

La formulación matemática del problema sería la siguiente:
\begin{align*}
\textrm{Maximizar}\ \ \     & \sum_{w\in W, c \in C}d_{w,c}x_{w,c}\\
\textrm{sujeto a}\ \ \      & \sum_{w\in W}x_{w,c}= 1 \, \, \forall c \in C \text{(Todos los clientes son servidos)}\\
                 \ \ \      & x_{w,c}\leq y_{w}  \, \, \forall w\in W, \forall c \in C \text{(Si el cliente $c$ es servido por $w$, el almacén debe ser seleccionado)}\\  
                 \ \ \      & \sum_{w\in W}y_{w}= P \, \,\text{(Seleccionar $P$ almacenes)}\\
                 \ \ \      & 0\leq x_{w,c}\leq 1, y_w\in \{0,1\} \\
                  \ \ \     & y_w\in \{0,1\} \\
\end{align*}

:::{exercise} Resolviendo el problema de localización de almacenes
:label: ex:pyomo2_almacen

Formula en Pyomo el problema de localización de almacenes y resuévelo considerando los siguientes datos:

![title](img/warehouse_data.png)

Puedes considerar como estructura de los datos la siguiente:

```{code-block} python
W = ['Harlingen', 'Memphis', 'Ashland']
C = ['NYC', 'LA', 'Chicago', 'Houston']
d = {('Harlingen', 'NYC'): 1956,
     ('Harlingen', 'LA'): 1606,
     ('Harlingen', 'Chicago'): 1410,
     ('Harlingen', 'Houston'): 330,
     ('Memphis', 'NYC'): 1096,
     ('Memphis', 'LA'): 1792,
     ('Memphis', 'Chicago'): 531,
     ('Memphis', 'Houston'): 567,
     ('Ashland', 'NYC'): 485,
     ('Ashland', 'LA'): 2322,
     ('Ashland', 'Chicago'): 324,
     ('Ashland', 'Houston'): 1236
    }
P = 2
```

````{solution} ex:pyomo2_almacen
:label: sol:pyomo2_almacen
:class: dropdown

```{code-block} python
import pyomo.environ as pe

# Datos
W = ['Harlingen', 'Memphis', 'Ashland']
C = ['NYC', 'LA', 'Chicago', 'Houston']
d = {('Harlingen', 'NYC'): 1956,
     ('Harlingen', 'LA'): 1606,
     ('Harlingen', 'Chicago'): 1410,
     ('Harlingen', 'Houston'): 330,
     ('Memphis', 'NYC'): 1096,
     ('Memphis', 'LA'): 1792,
     ('Memphis', 'Chicago'): 531,
     ('Memphis', 'Houston'): 567,
     ('Ashland', 'NYC'): 485,
     ('Ashland', 'LA'): 2322,
     ('Ashland', 'Chicago'): 324,
     ('Ashland', 'Houston'): 1236
    }
P = 2

# Creamos el modelo:
model = pe.ConcreteModel(name="(WL)")

# Variables:
model.x = pe.Var(W,C, bounds=(0,1))
model.y = pe.Var(W, within=pe.Binary)

# Función objetivo:
@model.Objective(sense=pyo.minimize)
def coste(model):
    return sum(d[w,c]*model.x[w,c] for w in W for c in C)
#model.obj = pe.Objective(
                expr=sum(d[w,c]*m.x[w,c] for w in W for c in C),
                sense= pe.minimize))

# Restricciones:
@model.Constraint(C)
def client_demand_rule(m, c):
    return sum(m.x[w,c] for w in W) == 1

@model.Constraint(W,C)
def warehouse_active_rule(m, w, c):
    return m.x[w,c] <= m.y[w]

@model.Constraint()
def num_warehouses_rule(m):
    return sum(m.y[w] for w in W) <= P
# model.num_warehouses_rule = pe.Constraint(expr=sum(m.y[w] for w in W) <= P)

# Resolvemos con HIGHS:
solver = 'appsi_highs'
SOLVER = pyo.SolverFactory(solver)
result_obj = SOLVER.solve(model, tee=True)

# Imprimimos el modelo:
#model.pprint()
model.display()
```

````

:::

In [37]:
# Tu código aquí

# Formulando de  manera abstracta: Abstract models

- Primero el modelo, y luego los datos.
- Contrucción en 2 pasos.
- Pyomo va almacenando las declaraciones básicas del modelo, pero no contruye de forma instantánea los objetos.
- En el momento de la creación del modelo, los datos son aplicados al conjunto de declaraciones abstractas para crear una instancia concreta del modelo.
- Favorece modelado más genérico y reutilización del modelo.
- Familiar para la gente acostumbrada a utilizar AMPL.

```{attention} 
A la hora de llevar a cabo la implementación, la única diferencia entre un `Concrete Model` y un `Abstract model` de Pyomo está en como definamos los conjuntos (`Set`) y los parámetros (`Param`). Si no inicializamos valores para ellos, entonces estaremos en el caso de un Abstract Model. 
```

```{important}💡 Fuentes de Datos: Pyomo AbstractModel

Existen tres vías para proporcionar datos a los Parámetros (`Param`) y Conjuntos (`Set`) del modelo abstracto:

1. 💾 Archivo .dat (Estilo AMPL)
   - Qué es: Archivo de texto plano con sintaxis estructurada.
   - Uso: Carga explícita de valores para Sets y Params durante la instanciación.

2. 📝 Declaración Explícita en `Param` (`initialize=`)
   - Qué es: Definición de valores fijos o funciones de inicialización directamente en el script de Python.
   - Ventaja: Ideal para datos constantes o lógica de cálculo de inicialización.

3. 🌐 Carga Externa (`load`)
   - Qué es: Módulos de Pyomo para integrar datos de fuentes dinámicas.
   - Fuentes: Excel, Bases de Datos (SQL).
   - Ventaja: Manejo de datasets grandes y variables.

🚨 Prioridad

El argumento `initialize=` en la declaración del componente anula (tiene prioridad sobre) cualquier dato provisto por fuentes externas (.dat, Excel, DB).
```

## Problema de la mochila

Vamos a mostrar cómo sería la implementación como Abstract model del problema de la mochila.

### Formulación del Modelo

In [38]:
import pyomo.environ as pe

model = pe.AbstractModel()

# Datos de forma abstracta (como conjuntos y parámetros)
model.ITEMS = pe.Set()
model.v = pe.Param( model.ITEMS, within=pe.PositiveReals )
model.w = pe.Param( model.ITEMS, within=pe.PositiveReals )
model.W_max = pe.Param( within=pe.PositiveReals )

# Variables
model.x = pe.Var( model.ITEMS, within=pe.Binary )

# Función objetivo
def value_rule(model):
    return sum( model.v[i]*model.x[i] for i in model.ITEMS )
model.value = pe.Objective( rule=value_rule, sense=pe.maximize )

# Restricción
def weight_rule(model):
    return sum( model.w[i]*model.x[i] for i in model.ITEMS )<= model.W_max
model.weight = pe.Constraint( rule=weight_rule )

In [39]:
model.pprint()

1 Set Declarations
    ITEMS : Size=0, Index=None, Ordered=Insertion
        Not constructed

3 Param Declarations
    W_max : Size=0, Index=None, Domain=PositiveReals, Default=None, Mutable=False
        Not constructed
    v : Size=0, Index=ITEMS, Domain=PositiveReals, Default=None, Mutable=False
        Not constructed
    w : Size=0, Index=ITEMS, Domain=PositiveReals, Default=None, Mutable=False
        Not constructed

1 Var Declarations
    x : Size=0, Index=ITEMS
        Not constructed

1 Objective Declarations
    value : Size=0, Index=None, Active=True
        Not constructed

1 Constraint Declarations
    weight : Size=0, Index=None, Active=True
        Not constructed

7 Declarations: ITEMS v w W_max x value weight


Como podemos ver, el modelo está definido pero de momento se encuentra **vacío** porque no tiene datos. Para poder crearlo debemos usar el método `create_instance` de Pyomo, al cual se le debe de pasar como argumento los datos de entrada de la instancia que se quiere resolver. Lo datos de entrada pueden pasarse en distintos formatos, tal y como puede ver en este [enlace](https://pyomo.readthedocs.io/en/stable/howto/abstract_models/data/index.html) de la documentación. A continuación mostraremos dos opciones:
 - Crear la instancia pasando los datos como un diccionario anidado de Python.
 - Crear la instancia a partir de un fichero de datos .dat.

### Instanciamos modelo abstracto usando un diccionario de Python  
https://pyomo.readthedocs.io/en/stable/howto/abstract_models/data/raw_dicts.html

In [40]:
data_input = {None:{
    'ITEMS': {None:['tornillo','candado','destornillador','toalla']}, 
    'v':{'tornillo': 8,'candado':3,'destornillador':6,'toalla':11},
    'w': {'tornillo': 5,'candado':7,'destornillador':4,'toalla':3},
    'W_max':{None: 14},
}}

In [41]:
instance = model.create_instance(data=data_input)
type(instance)

pyomo.core.base.PyomoModel.ConcreteModel

In [42]:
print('Model is contructed :',model.is_constructed())
print('Instance is contructed :',instance.is_constructed())

Model is contructed : False
Instance is contructed : True


In [43]:
instance.pprint()

1 Set Declarations
    ITEMS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'tornillo', 'candado', 'destornillador', 'toalla'}

3 Param Declarations
    W_max : Size=1, Index=None, Domain=PositiveReals, Default=None, Mutable=False
        Key  : Value
        None :    14
    v : Size=4, Index=ITEMS, Domain=PositiveReals, Default=None, Mutable=False
        Key            : Value
               candado :     3
        destornillador :     6
                toalla :    11
              tornillo :     8
    w : Size=4, Index=ITEMS, Domain=PositiveReals, Default=None, Mutable=False
        Key            : Value
               candado :     7
        destornillador :     4
                toalla :     3
              tornillo :     5

1 Var Declarations
    x : Size=4, Index=ITEMS
        Key            : Lower : Value : Upper : Fixed : Stale : Domain
               candado :     0 :  None :     1 : False :  T

In [44]:
# Resolvemos el modelo:
solver = 'appsi_highs'
SOLVER = pyo.SolverFactory(solver)
results = SOLVER.solve(instance, tee=True)

instance.display()

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 1 rows; 4 cols; 4 nonzeros; 4 integer variables (4 binary)
Coefficient ranges:
  Matrix [3e+00, 7e+00]
  Cost   [3e+00, 1e+01]
  Bound  [1e+00, 1e+00]
  RHS    [1e+01, 1e+01]
Presolving model
1 rows, 4 cols, 4 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0       

### Instanciamos modelo abstracto usando un archivo .dat  
* Se puede hacer utilizando lenguaje Python
* Se puede hacer una llamada a sistema a pyomo especificando los ficheros mochila.py y datos_input.dat

El fichero de datos tiene que tener un determinado formato, en función de cómo es el elemento que se quiere inicializar. Los detalles pueden verse en la documentación de Pyomo en este [enlace](https://pyomo.readthedocs.io/en/stable/howto/abstract_models/data/datfiles.html). En este caso, el fichero se llama `datos_input.dat` y tiene el siguiente aspecto:

```txt
set ITEMS := tornillo candado destornillador toalla ;
    
param: v w :=
    tornillo 8 5
    candado 3 7
    destornillador 6 4
    toalla 11 3;
    
param W_max := 14;
```

Lo resolvemos llamado al método `create_instance` especificando la ruta del fichero:

In [45]:
instance2 = model.create_instance(filename='datos_input.dat')
results = SOLVER.solve(instance2, tee=True)
instance2.display()

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 1 rows; 4 cols; 4 nonzeros; 4 integer variables (4 binary)
Coefficient ranges:
  Matrix [3e+00, 7e+00]
  Cost   [3e+00, 1e+01]
  Bound  [1e+00, 1e+00]
  RHS    [1e+01, 1e+01]
Presolving model
1 rows, 4 cols, 4 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0       

## 🧠 Ejercicios propuestos

:::{exercise} Problema de planificación como Abstract Model
:label: ex:pyomo2_abstractplan

Resuelve de nuevo el problema planificación pero formulándolo ahora como un Abstract Model de Pyomo y empleando un diccionario par pasarle los datos,

````{solution} ex:pyomo2_abstractplan
:label: sol:pyomo2_abstractplan
:class: dropdown

```{code-block} python
import pyomo.environ as pyo

# Definición del modelo abstracto
model = pyo.AbstractModel()

# Conjuntos
model.PRODUCTOS = pyo.Set()
model.RECURSOS = pyo.Set()

# Parámetros
model.demanda = pyo.Param(model.PRODUCTOS, default=None, mutable=True)
model.disponible = pyo.Param(model.RECURSOS, default=None, mutable=True)
model.cp = pyo.Param(model.PRODUCTOS)
model.cr = pyo.Param(model.RECURSOS)
model.a = pyo.Param(model.RECURSOS, model.PRODUCTOS)

# Variables
def cotas_x(model, recurso):
    return (0, model.disponible[recurso])
model.x = pyo.Var(model.RECURSOS, domain=pyo.NonNegativeReals, bounds=cotas_x)

def cotas_y(model, producto):
    return (0, model.demanda[producto])
model.y = pyo.Var(model.PRODUCTOS, domain=pyo.NonNegativeReals, bounds=cotas_y)

# Función objetivo
@model.Objective(sense=pyo.maximize)
def beneficio(model):
    ingresos = sum(model.cp[p] * model.y[p] for p in model.PRODUCTOS)
    costes = sum(model.cr[r] * model.x[r] for r in model.RECURSOS)
    return ingresos - costes

# Restricciones
@model.Constraint(model.RECURSOS)
def materiales_usados(model, r):
    return sum(model.a[r, p] * model.y[p] for p in model.PRODUCTOS) <= model.x[r]

# =======================
# Crear datos para la instancia
# =======================
data = {
    None: {
        'PRODUCTOS': {None: ['U', 'V']},
        'RECURSOS': {None: ['MP', 'MH A', 'MH B']},
        'demanda': {'U': 40, 'V': 1000},
        'disponible': {'MP': 1000, 'MH A': 80, 'MH B': 100},
        'cp': {'U': 270, 'V': 210},
        'cr': {'MP': 10, 'MH A': 50, 'MH B': 40},
        'a': {
            ('MP', 'U'): 10, ('MH A', 'U'): 1, ('MH B', 'U'): 2,
            ('MP', 'V'): 9,  ('MH A', 'V'): 1, ('MH B', 'V'): 1
        }
    }
}

# Crear la instancia del modelo abstracto
instance = model.create_instance(data)

# Resolver
solver = 'appsi_highs'
SOLVER = pyo.SolverFactory(solver)
results = SOLVER.solve(instance, tee=True)

# Mostrar resultados
print(f"Profit = {pyo.value(instance.beneficio)}")

print("\nProducción:")
for producto in instance.PRODUCTOS:
    print(f" {producto} producido = {pyo.value(instance.y[producto])}")

print("\nRecursos:")
for recurso in instance.RECURSOS:
    print(f" {recurso} consumido = {pyo.value(instance.x[recurso])}")

    
```

````

:::


In [46]:
# Tu código aquí